In [ ]:

from sklearn.neighbors import NearestNeighbors
eps = 0.5#0.05
min_samples = 4
data = np.array([[0, 0, 2], [1, 0, 0], [0, 0, 1], [0, 0, 1.4], [0, 0, 1.5], [0, 0, 1.6]])
neighbors_model = NearestNeighbors(n_neighbors=min_samples,radius=eps)
neighbors_model.fit(data)
distance_matric, idx = neighbors_model.kneighbors([[0, 0, 1.3]], return_distance=True)
print(distance_matric)
print(idx)
data[idx[distance_matric<eps]]
# idx[distance_matric>eps]
# This has worst case O(n^2) memory complexity
# neighborhoods = neighbors_model.radius_neighbors(data, return_distance=False)
# neighborhoods
# neighborhoods[1][0]
# np.array([len(neighbors) for neighbors in neighborhoods])
# A list of all core samples found.
#         core_samples = np.asarray(n_neighbors >= self.min_samples,
#                                   dtype=np.uint8)

In [ ]:
import numpy as np
import laspy
import cv2
from TronGisPy import GisIO, CRS, Normalizer, Interpolation
import CSF 
import os, sys, gc, subprocess # file system package
import pickle, json, shutil, itertools # file processing basic package
import laspy, PySaga, gdal # geo package
#import cv2 # open computer vision package
import pandas as pd
import geopandas as gpd
import numba
from scipy.stats import entropy
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon, MultiPolygon
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances, pairwise_distances_argmin
from util.las import *
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.ndimage.measurements import label

In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import laspy
import numpy as np


las_fp = "./data/123.las"

In [2]:
with laspy.file.File(las_fp, mode = "r") as lasFile:
    xs, ys, zs = lasFile.x, lasFile.y, lasFile.z
    intensities, classifications = lasFile.intensity.copy(), lasFile.Classification.copy()

In [3]:
# 隨便給點
x,y,z = (296087.44,2772992.78,23.76) 


In [4]:

reduce_area_x = (xs > x-1) & (xs < x+1)
reduce_area_y = (ys > y-1) & (ys < y+1)
reduce_area_z = (zs > z-1) & (zs < z+1)
xyz = np.array([xs, ys, zs]).T
reduce_area = reduce_area_x & reduce_area_y & reduce_area_z
data = np.array([xs[reduce_area], ys[reduce_area], zs[reduce_area]]).T
neighbors_model = NearestNeighbors()
neighbors_model.fit(data)
idx = neighbors_model.kneighbors([[x, y, z]], return_distance=False)
x_condi = (xs == data[idx[0][0],0])
y_condi = (ys == data[idx[0][0],1])
z_condi = (zs == data[idx[0][0],2])
init_pt = xyz[np.argwhere(x_condi&y_condi&z_condi)[0][0],:]
init_pt

array([2.96087445e+05, 2.77299278e+06, 2.37652497e+01])

In [8]:
%%time 
eps = 0.1#0.05
min_samples = 10
neighbors_model = NearestNeighbors(radius=eps, algorithm='kd_tree')
neighbors_model.fit(xyz)

visited_pts_idx = []
distance_matric, idx = neighbors_model.kneighbors([init_pt], return_distance=True)
pts_idx = list(idx)
xyz[idx[distance_matric<eps]]

Wall time: 44.2 s


array([[2.96087445e+05, 2.77299278e+06, 2.37652497e+01],
       [2.96087432e+05, 2.77299279e+06, 2.37517497e+01],
       [2.96087461e+05, 2.77299278e+06, 2.37804997e+01],
       [2.96087423e+05, 2.77299279e+06, 2.37784997e+01],
       [2.96087471e+05, 2.77299278e+06, 2.37514997e+01]])

In [10]:
tmp

array([[2.96087456e+05, 2.77299278e+06, 2.37377497e+01],
       [2.96087445e+05, 2.77299278e+06, 2.37652497e+01],
       [2.96087406e+05, 2.77299279e+06, 2.37624997e+01],
       [2.96087432e+05, 2.77299279e+06, 2.37517497e+01],
       [2.96087461e+05, 2.77299278e+06, 2.37804997e+01]])

In [9]:
tmp == xyz[idx[distance_matric<eps]]

array([[False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False]])

In [ ]:
def RangeQuery(X, Q, eps):
    neighbors = list()
    for pt in xyz:
        

In [ ]:
visited_pts_idx = []
distance_matric, idx = neighbors_model.kneighbors([init_pt], return_distance=True)
pts_idx = list(idx)
# print(distance_matric)
# print(idx)
xyz[idx[distance_matric<eps]]

In [ ]:
idx[distance_matric<eps]

In [ ]:
xyz[14966635,:]
neighbors_model.kneighbors([xyz[14966635,:]], return_distance=True)

In [8]:

class Point(object):
    ''' internal helper class to support algorithm implementation'''
    def __init__(self,feature_vector):
        # feature vector should be something like a list or a numpy
        # array
        self.feature_vector = feature_vector
        self.cluster = None
        self.visited = False

    def __str__(self):
        return str(self.feature_vector)

def _as_points(points):
    ''' convert a list of list- or array-type objects to internal
    Point class'''
    return [Point(point) for point in points]

def as_lists(clusters):
    ''' converts the Points in each cluster back into regular feature
    vectors (lists).'''
    clusters_as_points = {}
    for cluster, members in clusters.items():
        clusters_as_points[cluster] = [member.feature_vector for member in members]
    return clusters_as_points

def print_points(points):
    ''' a wierd klugey function for printing lists of points. ''' 
    s = ''
    for p in points:
        s += str(p) + '\n'
    return s[:-2]

def euclidean(x,y):
    ''' calculate the euclidean distance between x and y.'''
    # np.sqrt((x0-y0)^2 + ... (xN-yN)^2)
    assert len(x) == len(y)
    sum = 0.0
    for i in range(len(x)):
        sum += np.power(x[i] - y[i],2)
    return np.sqrt(sum)

def immediate_neighbours(point, all_points, epsilon, distance, debug):
    ''' find the immediate neighbours of point.'''
    # XXX TODO: this is probably a stupid way to do it; if we could
    # use a grid approach it should make this much faster.
    neighbours = []
    for p in all_points:
        if p == point:
            # you cant be your own neighbour...!
            continue
        d = distance(point.feature_vector,p.feature_vector)
        if d < epsilon:
            neighbours.append(p)
    return neighbours

def add_connected(points, all_points, epsilon, min_pts, current_cluster, distance, debug):
        ''' find every point in the set of all_points which are
        density-connected, starting with the initial points list. '''
        cluster_points = []
        for point in points:
            if not point.visited:
                point.visited = True
                new_points = immediate_neighbours(point, all_points, epsilon, distance, debug)
                if len(new_points) >= min_pts:                                
                    # append any new points on the end of the list we're
                    # already iterating over.
                    for p in new_points:
                        if p not in points:
                            points.append(p)

            # here, we separate 'visited' from cluster membership, since
            # 'visited' only helps keep track of if we've checked this
            # point for neighbours. it may or may not have been assessed
            # for cluster membership at that point.
            if not point.cluster:
                cluster_points.append(point)
                point.cluster = current_cluster
        if debug: 
            print('Added points %s' % print_points(cluster_points))
        return cluster_points

def dbscan(points, epsilon, min_pts, distance=euclidean, debug=False):
    assert isinstance(points, list)
    epsilon = float(epsilon)
    if not isinstance(points[0], Point):
        # only check the first list instance. imperfect, but the lists
        # could be arbitrarily long.
        points = _as_points(points)

    if debug:
        print('\nEpsilon: %.2f  Min_Pts: %d' % (epsilon, min_pts))

    clusters = {}     # each cluster is a list of points
    clusters[-1] = [] # store all the points deemed noise here. 
    current_cluster = -1
    for point in points:
        if not point.visited:
            point.visited = True
            neighbours = immediate_neighbours(point, points, epsilon, distance, debug)
            if len(neighbours) >= min_pts:
                current_cluster += 1
                if debug: 
                    print('\nCreating new cluster %d' % (current_cluster))
                    print('%s' % str(point))
                point.cluster = current_cluster                
                cluster = [point,]
                cluster.extend(add_connected(neighbours, points, epsilon, min_pts, 
                                             current_cluster, distance, debug))
                clusters[current_cluster] = cluster
                break
            else:
                clusters[-1].append(point)
                if debug: 
                    print('\nPoint %s has no density-connected neighbours.' % str(point.feature_vector))

    # return the dictionary of clusters, converting the Point objects
    # in the clusters back to regular lists
    return as_lists(clusters)

if __name__ == '__main__':

    import random

    epsilon = 2.0
    min_pts = 2.0
    points = []
    points.append([1,1])
    points.append([1.5,1])
    points.append([1.8,1.5])
    points.append([2.1,1])
    points.append([3.1,2])
    points.append([4.1,2])
    points.append([5.1,2])
    points.append([10,10])
    points.append([11,10.5])
    points.append([9.5,11])
    points.append([9.9,11.4])
    points.append([15.0, 17.0])
    points.append([15.0, 17.0])
    points.append([7.5, -5.0])
    points.append([57.5, -55.0])
    clusters = dbscan(points, epsilon, min_pts, debug=True)
    print('\n========== Results of Clustering =============')
    print("Numbers of all points: %d" % len(points))
    for cluster, members in clusters.items():
        if cluster == -1:
            print('\n--------Noises  %d---------' % cluster)
        else:
            print('\n--------Cluster %d---------' % cluster)
        for point in members:
            print(point)

#     points = []
#     for i in range(100):
#         points.append([random.uniform(0.0, 20.0), random.uniform(0.0, 20.0), random.uniform(0.0, 20.0)])

#     clusters = dbscan(points, epsilon, min_pts, debug=True)
#     print('\n========== Results of Clustering =============')
#     for cluster, members in clusters.items():
#         print('\n--------Cluster %d---------' % cluster)
#         for point in members:
#             print(point)



Epsilon: 2.00  Min_Pts: 2

Creating new cluster 0
[1, 1]
Added points [1.5, 1]
[1.8, 1.5]
[2.1, 1]
[1, 1]
[3.1, 2]
[4.1, 2]
[5.1, 2

========== Results of Clustering =============
Numbers of all points: 15

--------Noises  -1---------

--------Cluster 0---------
[1, 1]
[1.5, 1]
[1.8, 1.5]
[2.1, 1]
[1, 1]
[3.1, 2]
[4.1, 2]
[5.1, 2]


In [ ]:
import scipy
from scipy.spatial.kdtree import KDTree
%%time

# Build the KD Tree
tree = KDTree(xyz)
# This should do the same as the FLANN example above, though it might
# be a little slower.
tree.query(xyz[14966635,], k = eps)

In [10]:
xyz[14966635,]==init_pt

array([ True,  True,  True])

In [ ]:
tree.data[:10]

In [18]:
N_sub

13500.0